In [1]:
#please note, you can only use this in the jupyter and not anywhere else e.g. anaconda lab

In [2]:
%load_ext autoreload

%autoreload 2
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.callbacks as cb
import time
import numpy as np
from data import get_datasets
import matplotlib.pyplot as plt
import os
import shutil
from functools import partial
from data import get_datasets
from experiment import create_model
from sentiment import sentiment
import model_adv

In [3]:
(train_dataset, test_dataset), info = get_datasets()
encoder = info.features['text'].encoder
vocab_size=info.features['text'].encoder.vocab_size

In [4]:
transformer, _optimizer, _checkpoint, _manager = create_model(models_dir="models/pos_enc_True", load_checkpoint=True, 
                           vocab_size=vocab_size, use_positional_encoding=True, run_eagerly=True)

Loaded previously trained model.


In [5]:
def sentiment(inp_sentence, encoder, transformer,adv_k = None):
    # inp sentence is the review  

    
    inp_sentence = encoder.encode(inp_sentence) 

    

    encoder_input = tf.expand_dims(inp_sentence, 0)
    if adv_k is not None:
        predictions,weights,_ = transformer(encoder_input,custom_k=adv_k ,training=False)
    else:
        predictions,weights,_ = transformer(encoder_input ,training=False)
    
    
    sent = tf.squeeze(predictions, axis=0)
    if sent >= 0.5:
        sent = 'pos'
    else:
        sent = 'neg'
    return sent,weights
    

In [6]:
from bertviz import head_view
import torch

In [7]:

%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [8]:
sentence = "This is a very terrible movie . I will never watch this bad movie again ."

tokens = sentence.split()

prediction,weights = sentiment(sentence, encoder=encoder, transformer=transformer)

In [9]:
head_view(torch.tensor(weights.numpy()[0]),tokens)
print("prediction after permutation is %str"%prediction)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

prediction after permutation is negtr
